In [2]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import scipy
from itertools import permutations

from scipy.sparse import coo_matrix
from google.cloud import storage

import json 
import datetime as dt


ModuleNotFoundError: No module named 'pyspark'

In [3]:
#  Create the file list 
client = storage.Client()
bucket = client.bucket('bgse-datawarehousing-random-tweets')
jsons_list = []
for blob in bucket.list_blobs(prefix = ''):
    jsons_list.append('gs://bgse-datawarehousing-random-tweets/'+blob.name)

#  How many of the datasets I want to read
file_no = 5

#  Convert the first file_no of the JSON files into a Spark dataframe
dataset = spark.read.json(jsons_list[0:file_no])

#  Extract the list of all the possible permutations of 2 hashtags from each tweet, then reduced to get the # of 
#  Times each permutation happens across all tweets
hash_perm = dataset.rdd \
        .filter(lambda r: ((r.entities is not None) and (len(r.entities.hashtags) > 1))) \
        .map(lambda r: [i.text.lower() for i in r.entities.hashtags]) \
        .flatMap(lambda r: permutations(r, 2)) \
        .map(lambda r: (r, 1)) \
        .reduceByKey(lambda a, b: a + b) \
        .map(lambda r: (r[0][0], r[0][1], r[1])) \
        .collect()


#  Create the dictionary with all the unique hashtags in the tweets
hashtags = sorted(list(set([x[0] for x in hash_perm])))
dict_hash = {hashtags[i] : i for i in range(0, len(hashtags))}

#  Create rows, columns and data
row = np.array([dict_hash[i[0]] for i in hash_perm])
column = np.array([dict_hash[i[1]] for i in hash_perm])
data = np.array([i[2] for i in hash_perm])
# Create the sparse matrix
matrix_hashtags = coo_matrix((data, (row,column)),
                    shape=(len(dict_hash),
                           len(dict_hash)))
#  Save it to a file
scipy.sparse.save_npz("matrix_hashtags.npz", matrix_hashtags, compressed=True)

NameError: name 'storage' is not defined